In [4]:
# Visualize Docker Compose Topology with sPyTial

import yaml
from spytial import diagram
from dataclasses import dataclass
from typing import List, Optional
from io import StringIO

@dataclass
class ServiceNode:
    name: str
    image: str
    ports: List[str] = None
    networks: List[str] = None
    depends_on: List[str] = None

    def __post_init__(self):
        if self.ports is None:
            self.ports = []
        if self.networks is None:
            self.networks = []
        if self.depends_on is None:
            self.depends_on = []

# Mock docker-compose.yml as a string
mock_compose = """
services:
  web:
    image: nginx
    ports:
      - "80:80"
    networks:
      - frontend
    depends_on:
      - db
  db:
    image: postgres
    networks:
      - backend
  api:
    image: python:3.9
    ports:
      - "5000:5000"
    networks:
      - frontend
      - backend
    depends_on:
      - db

networks:
  frontend:
  backend:
"""

# Parse the mock YAML
compose_data = yaml.safe_load(StringIO(mock_compose))
services = []
for name, config in compose_data.get('services', {}).items():
    services.append(ServiceNode(
        name=name,
        image=config.get('image', ''),
        ports=config.get('ports', []),
        networks=config.get('networks', []),
        depends_on=config.get('depends_on', [])
    ))

# Visualize the services
diagram(services)  # Inline visualization of Docker Compose topology

# PYTHON ON WHALES

In [ ]:
from python_on_whales import docker
from spytial import diagram

def compose_live(project=None):
    # list containers of a compose app; you can also pass compose files to docker.compose
    procs = docker.compose.ps() if project is None else docker.compose.ps(project_name=project)
    nodes, edges = {}, []

    for p in procs:
        # p is a Container object with attributes mirroring `docker ps/inspect`
        sid = f"service:{p.labels.get('com.docker.compose.project','?')}:{p.labels.get('com.docker.compose.service', p.name)}"
        nodes.setdefault(sid, {"kind":"Service","name":p.labels.get('com.docker.compose.service', p.name), "attrs":{}})
        nodes[f"container:{p.id[:12]}"] = {"kind":"Container","name":p.name,"attrs":{"status":p.state.status}}
        edges.append(("runs", sid, f"container:{p.id[:12]}"))
        # networks & ports available via p.networks, p.ports, etc.
    return {"nodes": list(nodes.values()), "edges": edges}

# Visualize the live compose topology
data = compose_live()
diagram(data)

ModuleNotFoundError: No module named 'python_on_whales'